In [1]:
# setup
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt 
import requests
import calendar
import geopandas as gpd
import os.path as os
import scipy.stats
import seaborn.palettes
import seaborn.utils
import sys
from census import Census
from us import states
import http.client, urllib.request, urllib.parse, urllib.error, base64
import config

root= r"C:/Users/Jennah/Desktop/Code/machine-learning-final"
inp= os.join(root, "data", "1_raw")
out= os.join(root, "data", "2_intermediate")

In [47]:
import psycopg2 as psy
# load furman db
connection = psy.connect(
    host="fcdata.c7h93yqbqnvu.us-east-1.rds.amazonaws.com",
    port=5432,
    user = "furmandata",
    password = config.furman_pwd,
    database = "fcdata")
cursor = connection.cursor()

In [49]:
## load pluto data as points
pluto= gpd.GeoDataFrame.from_postgis("""SELECT bbl, 
                                               tract2010,
                                               geom_point FROM mappluto_21v3
                                        WHERE landuse IN ('01', '02', '03', '04')""", connection,
                                    geom_col = "geom_point")

In [8]:
from pydrive.auth import GoogleAuth

gauth = GoogleAuth()
gauth.LocalWebserverAuth() # Creates local webserver and auto handles authentication.

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=570719947456-53haph7mjc317oel1ujh0kmjbfd499jf.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=offline&response_type=code

Authentication successful.


In [9]:
from pydrive.drive import GoogleDrive
drive = GoogleDrive(gauth)
fileList = drive.ListFile({'q': "'1kiYbPtSYavHQKzyMWtXaLhP-JM2toXRm' in parents and trashed=false"}).GetList()
for file in fileList:
  print('Title: %s, ID: %s' % (file['title'], file['id']))
  # Get the folder ID that you want
  if(file['title'] == "To Share"):
      fileID = file['id']

Title: fire_box_sum.csv, ID: 1zYSWlWtgNWsruaEkSSCqyXHIIdq7BEAP
Title: acs_2009_2019.csv, ID: 17PvjIPesTzugausfzMi7rTiXahTiHaH1
Title: census_acs, ID: 1DjJ3pOLAo4MOpgmeNPSXJbRzF_S10fDM
Title: weighted_borobox.csv, ID: 1H8yM7UH14YdMqi5ASjHMd_dwA4C7-lKe
Title: pluto_borobox.csv, ID: 1Yf4xIIOBmKU2WIlImat8qaN3z2L239Nw
Title: fire_dispatch_tracts.csv, ID: 1psGrpBmGmNHPwM4lsRyiDX9tX5cf_UsO
Title: pluto_borobox.geojson, ID: 1UGelHNCzfSwsvbqBzyBX5GkrY_PmPGdp
Title: fire_dispatch.csv, ID: 1ulJlwpaezzq_aYV6jZzRWpmXLVlpO8qu
Title: ct_box_xwalk.csv, ID: 1nW-ODlhdUFKhTYzHZQOu3CCZN87oYXel
Title: dev_permits.csv, ID: 1mH_X0bLa_a1r-hpzwz43HpC7fFcayufp
Title: all_vacate.csv, ID: 1WfnTEjOaDNEb3E2vurXS_XkxJ0J1isdN
Title: pluto_df.csv, ID: 1j2KR6Tpa4CTA0k1omVfE52UYlMBMz17o
Title: pluto_ct.csv, ID: 1ZLZZS79fi4Z0RFTNMRO6GfkNwughRuej
Title: ll84_avgenergy.csv, ID: 1NOKtbGyL3_TBSMp_eqybVaQc67t33yuo
Title: fire_vacate.csv, ID: 11LV7007TbrVlfilkSvslDewQRZ9m0F-A
Title: speculation_hpd.csv, ID: 1yf86QR6yv2Z9Q6WNIN

## Calculate catchment for alarm boxes

In [ ]:
firecomp= gpd.read_file("https://services5.arcgis.com/GfwWNkhOj9bNBqoJ/arcgis/rest/services/NYC_Fire_Companies/FeatureServer/0/query?where=1=1&outFields=*&outSR=4326&f=pgeojson")
firecomp

firebat= gpd.read_file("https://services5.arcgis.com/GfwWNkhOj9bNBqoJ/arcgis/rest/services/NYC_Fire_Battalions/FeatureServer/0/query?where=1=1&outFields=*&outSR=4326&f=pgeojson")
firebat

firediv= gpd.read_file("https://services5.arcgis.com/GfwWNkhOj9bNBqoJ/arcgis/rest/services/NYC_Fire_Divisions/FeatureServer/0/query?where=1=1&outFields=*&outSR=4326&f=pgeojson")
firediv

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))
firebat.plot(ax=ax, color='#6a6a6a', edgecolor='white', linewidth=2)
firediv.plot(ax=ax, color='none', edgecolor='red')

In [ ]:
firevac= pd.read_csv("https://data.cityofnewyork.us/resource/n5xc-7jfa.csv?$limit=100000")
firevac.shape

In [ ]:
alarmbox= gpd.read_file("https://data.cityofnewyork.us/resource/v57i-gtxb.geojson?$limit=100000").to_crs(2263)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))
firebat.to_crs(2263).plot(ax=ax, color='#6a6a6a', edgecolor='white', linewidth=2)
firediv.to_crs(2263).plot(ax=ax, color='none', edgecolor='red')
alarmbox.plot(ax=ax, color='green', markersize=0.5, alpha = 0.8)

In [ ]:
pluto.head()
#alarmbox

In [ ]:
from scipy.spatial import cKDTree
from shapely.geometry import Point

def ckdnearest(gdA, gdB):

    nA = np.array(list(gdA.geometry.apply(lambda x: (x.x, x.y))))
    nB = np.array(list(gdB.geometry.apply(lambda x: (x.x, x.y))))
    btree = cKDTree(nB)
    dist, idx = btree.query(nA, k=1)
    gdB_nearest = gdB.iloc[idx].drop(columns="geometry").reset_index(drop=True)
    gdf = pd.concat(
        [
            gdA.reset_index(drop=True),
            gdB_nearest,
            pd.Series(dist, name='dist')
        ], 
        axis=1)

    return gdf

pluto_borobox= ckdnearest(pluto, alarmbox)

In [ ]:
pluto_borobox.head()

In [ ]:
pluto_borobox.to_csv(os.join(out, "pluto_borobox.csv"))
#pluto_borobox.to_file(os.join(out, "pluto_borobox.geojson"), driver = "GeoJSON")

## Create Census Tract Weights

In [3]:
pluto_borobox = gpd.read_file(os.join(out, "pluto_borobox.geojson"))

In [53]:
plutobox_ct= pd.merge(pluto_borobox.loc[:, ["bbl", "location", "borobox", "box_type", "latitude", "longitude"]],
         pluto.loc[:, ["bbl", "tract2010"]], on = "bbl")

plutobox_ct['boroct2010']= plutobox_ct['bbl'].str[0] + plutobox_ct['tract2010'].astype(str).str.replace(".", "", regex = False).str.pad(width = 6, side = "right",
                                                                                                                      fillchar = "0")
plutobox_ct.head()

,bbl,location,borobox,box_type,latitude,longitude,tract2010,boroct2010
0,3042270037,BELMONT AVE & MILFORD ST,B1826,ERS,40.67338539,-73.87680041,1194,3119400
1,3042670001,BLAKE AVE & EUCLID AVE,B1838,ERS,40.67132686,-73.87092539,1208,3120800
2,3055400063,20 AVE & 63 ST,B2859,ERS,40.61768416,-73.98465158,0248,3024800
3,3029077506,MASPETH AVE & CONSELYEA ST,B0184,BARS,40.71551913,-73.94122766,0477,3047700
4,3055440004,15 AVE & 67 ST,B2805,ERS,40.62215491,-73.9983325,0250,3025000


In [113]:
# load acs data (to check census tract recoding)
csv = drive.CreateFile({'id': '17PvjIPesTzugausfzMi7rTiXahTiHaH1'})
csv.GetContentFile('acs_data.csv')  

acs= pd.read_csv('acs_data.csv').drop(['Unnamed: 0', 'Unnamed: 0.1'], axis = 1)
acs= acs.loc[acs['year']==2019]
acs.head()

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=570719947456-53haph7mjc317oel1ujh0kmjbfd499jf.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=offline&response_type=code

Authentication successful.


,geofips,totpop,popdens,areasqmile,males,females,pop18under,pop18_34,pop35_64,pop65plus,...,pctnhblack,pctnhaian,pctnhasian,pctnhhawainpi,pctnhother,pctnhtwoplus,pctownerocch,pctrentocch,crowding,crowdingrenter
21721,36005000100,6864,10599.550,0.647574,6345,519,103,3322,3392,47,...,52.462121,0.262238,2.636946,0.0,0.276807,0.509907,NaN,NaN,NaN,NaN
21722,36005000200,4532,24481.190,0.185122,2193,2339,1134,862,1605,931,...,20.300088,0.000000,1.654898,0.0,1.323919,0.000000,52.459016,47.540984,11.997019,18.181818
21723,36005000400,5516,18537.140,0.297565,2810,2706,1071,1319,2362,764,...,22.915156,0.000000,2.193619,0.0,0.163162,1.341552,79.040157,20.959843,1.860921,4.906542
21724,36005001600,5825,31101.490,0.187290,2422,3403,1546,1399,1978,902,...,34.695279,0.000000,1.098712,0.0,0.858369,0.429185,12.430238,87.569762,11.415525,12.514484
21725,36005001900,3141,4877.152,0.644023,1583,1558,898,1052,1111,80,...,29.831264,0.000000,2.292264,0.0,0.700414,1.782872,10.565338,89.434662,9.638554,10.777202


In [132]:
acs['geofips']= acs['geofips'].astype(str).str.replace('36', '')
acs['geofips_recode']= np.select([acs['geofips'].str[0:3] == '005',
                                 acs['geofips'].str[0:3] == '047',
                                 acs['geofips'].str[0:3] == '061',
                                 acs['geofips'].str[0:3] == '081',
                                 acs['geofips'].str[0:3] == '085'],
                                ['2' + acs['geofips'].str[3:],
                                '3' + acs['geofips'].str[3:],
                                 '1' + acs['geofips'].str[3:],
                                 '4' + acs['geofips'].str[3:],
                                 '5' + acs['geofips'].str[3:]])

print(acs.loc[acs['geofips_recode'].str.len() < 7, 'geofips_recode'])
acs.loc[acs['geofips_recode'].str.len() < 7, 'geofips_recode']= \
acs.loc[acs['geofips_recode'].str.len() < 7, 'geofips_recode'].str.pad(width = 7, fillchar = "0", side = "right")

acs[['geofips_recode', 'geofips']].head()

21886    20200
21958    20300
21972    20000
21973    20100
21974    20300
21975    20400
21976    20501
21977    20502
21978    20700
21979    20800
21980    20901
21981    20902
22044    20400
22081    30000
22157    30100
22248    30200
22340    30300
22361    30001
22362    30002
22363    30100
22364    30200
22365    30300
22366    30400
22367    30501
22368    30502
22369    30600
22370    30700
22371    30900
22435    30400
22586    30600
22628    30700
22678    30800
22723    30900
22856    10001
22857    10002
22957    10100
23065    10200
23132    40000
23209    40100
23280    40200
23352    40100
23353    40300
23354    40500
23355    40600
23356    40700
23357    40800
23466    40501
23742    41700
23792    50000
Name: geofips_recode, dtype: object


,geofips_recode,geofips
21721,2000100,005000100
21722,2000200,005000200
21723,2000400,005000400
21724,2001600,005001600
21725,2001900,005001900


In [133]:
# this is not true
# assert (acs["geofips"].str.len() == 9).all()
assert (acs["geofips_recode"].str.len() == 7).all()
print(acs.loc[acs["geofips"].str.len() != 9].shape[0])
acs.loc[acs["geofips"].str.len() != 9].head()

49


,geofips,totpop,popdens,areasqmile,males,females,pop18under,pop18_34,pop35_64,pop65plus,...,pctnhaian,pctnhasian,pctnhhawainpi,pctnhother,pctnhtwoplus,pctownerocch,pctrentocch,crowding,crowdingrenter,geofips_recode
21886,0050200,1816,51082.47,0.035550,811,1005,541,592,482,201,...,0.000000,10.848018,0.000000,0.000000,0.055066,20.683761,79.316239,9.401709,9.913793,2020000
21958,0050300,6714,94303.96,0.071195,2767,3947,1777,1859,2504,574,...,0.000000,3.217158,0.000000,0.148943,0.000000,17.425968,82.574032,9.719058,11.770115,2030000
21972,0050000,3271,31828.00,0.102771,1626,1645,648,650,1322,651,...,0.825436,10.394375,0.000000,0.000000,0.917151,73.834746,26.165254,5.932203,14.574899,2000000
21973,0050100,5576,87614.04,0.063643,2164,3412,1354,1460,1753,1009,...,0.000000,0.053802,0.000000,0.000000,0.000000,1.481481,98.518519,4.768519,4.840226,2010000
21974,0050300,7517,87908.98,0.085509,3825,3692,1957,2334,2576,650,...,0.000000,0.000000,1.277105,0.332579,0.266064,7.024467,92.975533,14.640884,14.855688,2030000


In [134]:
# merge pluto (w/ borobox and census tract to ACS data with recoded geofips var)
pluto_acs= plutobox_ct.merge(acs, left_on = "boroct2010", right_on = "geofips_recode", 
                             how = "outer",
                             indicator = True)
pluto_acs.head()

,bbl,location,borobox,box_type,latitude,longitude,tract2010,boroct2010,geofips,totpop,...,pctnhasian,pctnhhawainpi,pctnhother,pctnhtwoplus,pctownerocch,pctrentocch,crowding,crowdingrenter,geofips_recode,_merge
0,3042270037,BELMONT AVE & MILFORD ST,B1826,ERS,40.67338539,-73.87680041,1194,3119400,047119400,4892.0,...,1.287817,0.0,0.0,0.0,31.165919,68.834081,33.781764,44.082519,3119400,both
1,3040220001,BELMONT AVE & ATKINS AVE,B1804,ERS,40.67311965,-73.87864502,1194,3119400,047119400,4892.0,...,1.287817,0.0,0.0,0.0,31.165919,68.834081,33.781764,44.082519,3119400,both
2,3040220005,BELMONT AVE & ATKINS AVE,B1804,ERS,40.67311965,-73.87864502,1194,3119400,047119400,4892.0,...,1.287817,0.0,0.0,0.0,31.165919,68.834081,33.781764,44.082519,3119400,both
3,3040220006,BELMONT AVE & ATKINS AVE,B1804,ERS,40.67311965,-73.87864502,1194,3119400,047119400,4892.0,...,1.287817,0.0,0.0,0.0,31.165919,68.834081,33.781764,44.082519,3119400,both
4,3040220119,PITKIN AVE & BERRIMAN ST,B1824,ERS,40.67425552,-73.8798826,1194,3119400,047119400,4892.0,...,1.287817,0.0,0.0,0.0,31.165919,68.834081,33.781764,44.082519,3119400,both


In [135]:
# check merge results
pluto_acs["_merge"].value_counts()

both          761465
left_only      14860
right_only        79
Name: _merge, dtype: int64

In [136]:
pluto_acs.loc[pluto_acs["_merge"]=="right_only", "geofips_recode"].value_counts()
# note: bronx tract 001 is Rikers Island
# i think these are ok, spot checked some--many seem like parks
# top two with population seem like rikers and part of roosevelt island?
pluto_acs.loc[pluto_acs["_merge"]=="right_only", :].sort_values('totpop', ascending = False)

,bbl,location,borobox,box_type,latitude,longitude,tract2010,boroct2010,geofips,totpop,...,pctnhasian,pctnhhawainpi,pctnhother,pctnhtwoplus,pctownerocch,pctrentocch,crowding,crowdingrenter,geofips_recode,_merge
776325,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,005000100,6864.0,...,2.636946,0.0,0.276807,0.509907,NaN,NaN,NaN,NaN,2000100,right_only
776364,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,061023802,6086.0,...,26.043378,0.0,0.000000,2.678278,4.954955,95.045045,9.966216,10.189573,1023802,right_only
776397,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0811700,5846.0,...,47.006500,0.0,0.000000,0.530277,76.128216,23.871784,4.765922,3.533569,4170000,right_only
776333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0050100,5576.0,...,0.053802,0.0,0.000000,0.000000,1.481481,98.518519,4.768519,4.840226,2010000,right_only
776336,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0050501,3901.0,...,0.000000,0.0,0.000000,0.487055,8.842559,91.157441,14.018692,11.041009,2050100,right_only
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
776371,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,081009900,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4009900,right_only
776372,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,081010701,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4010701,right_only
776373,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,081022900,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4022900,right_only
776375,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,081024600,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4024600,right_only


In [138]:
pluto_acs.loc[pluto_acs["_merge"]=="left_only", "boroct2010"].value_counts().head()

3033600    913
4136700    739
4036600    651
5003600    638
3013600    583
Name: boroct2010, dtype: int64

In [142]:
acs[acs['geofips_recode'].str.contains('336')]

,geofips,totpop,popdens,areasqmile,males,females,pop18under,pop18_34,pop35_64,pop65plus,...,pctnhaian,pctnhasian,pctnhhawainpi,pctnhother,pctnhtwoplus,pctownerocch,pctrentocch,crowding,crowdingrenter,geofips_recode


In [161]:
# comparison census tracts, check it merges with comparison census tracts
ct_comp= pd.read_csv(os.join(inp, "NYC_Census_Tracts_for_2010_US_Census.csv"))
ct_comp["BoroCT2010"]= ct_comp["BoroCT2010"].astype(str)

ct_comp_merge= plutobox_ct.merge(ct_comp, left_on = "boroct2010", right_on = "BoroCT2010",
                                 how = "outer",
                                 indicator = True)
ct_comp.head()
ct_comp_merge["_merge"].value_counts()

both          764555
right_only        55
left_only          5
Name: _merge, dtype: int64

In [162]:
ct_comp.loc[ct_comp["BoroCT2010"].astype(str).str.contains('336')]

,OBJECTID,CTLabel,BoroCode,BoroName,CT2010,BoroCT2010,CDEligibil,NTACode,NTAName,PUMA,Shape__Area,Shape__Length
907,908,336.0,2,Bronx,33600,2033600,E,BX07,Bronxdale,3704,1.985238e+06,6220.861268
1882,1883,336.0,3,Brooklyn,33600,3033600,I,BK21,Seagate-Coney Island,4018,7.258682e+06,10967.801773


## Count of buildings by census tract

In [150]:
plutobox_ct_count= plutobox_ct.value_counts(['boroct2010', 'borobox']).rename('count').reset_index()
plutobox_ct_count

,boroct2010,borobox,count
0,3070201,B4116,1614
1,3033600,B3520,836
2,2051600,X4607,694
3,5020803,R4329,661
4,5017007,R3387,660
...,...,...,...
18626,3031300,B1579,1
18627,3031900,B1044,1
18628,3032500,B3762,1
18629,3032700,B3762,1


In [151]:
plutobox_ct_count= pd.merge(plutobox_ct_count,
         plutobox_ct_count.groupby('boroct2010', as_index = False)["count"].sum()\
         .rename({'count':'total'}, axis = 1), on = "boroct2010")
plutobox_ct_count['wgt']= plutobox_ct_count['count'].div(plutobox_ct_count['total'])

In [152]:
plutobox_ct_count

,boroct2010,borobox,count,total,wgt
0,3070201,B4116,1614,1876,0.860341
1,3070201,B3994,228,1876,0.121535
2,3070201,B4103,34,1876,0.018124
3,3033600,B3520,836,913,0.915663
4,3033600,B3522,77,913,0.084337
...,...,...,...,...,...
18626,3008500,B3829,1,4,0.250000
18627,3035000,B3562,1,2,0.500000
18628,3035000,B3564,1,2,0.500000
18629,3035601,B3387,1,1,1.000000


In [153]:
## save census tract, borobox, weight crosswalk
plutobox_ct_count.to_csv(os.join(out, "ct_box_xwalk.csv"))